# Visual Data Analysis of Fraudulent Transactions

Your CFO has also requested detailed trends data on specific card holders. Use the starter notebook to query your database and generate visualizations that supply the requested information as follows, then add your visualizations and observations to your markdown report.

In [10]:
# Initial imports
import pandas as pd
import datetime
import calendar
import hvplot.pandas
from sqlalchemy import create_engine

In [2]:
# Create a connection to the database
engine = create_engine("postgresql://postgres:postgres @localhost:5432/fraud-detection")


## Data Analysis Question 1

The two most important customers of the firm may have been hacked. Verify if there are any fraudulent transactions in their history. For privacy reasons, you only know that their cardholder IDs are 2 and 18.

* Using hvPlot, create a line plot representing the time series of transactions over the course of the year for each cardholder separately. 

* Next, to better compare their patterns, create a single line plot that containins both card holders' trend data.  

* What difference do you observe between the consumption patterns? Does the difference suggest a fraudulent transaction? Explain your rationale in the markdown report.

In [3]:
# loading data for card holder 2 and 18 from the database
# Write the query

query = """
SELECT card_holder.cardholder_id, transaction.date, transaction.amount
FROM transaction
JOIN credit_card ON transaction.card = credit_card.card 
JOIN card_holder ON credit_card.cardholder_id = card_holder.cardholder_id
WHERE card_holder.cardholder_id  = '2' 
OR card_holder.cardholder_id  = '18'
ORDER BY transaction.date;
        """
# Create a DataFrame from the query result. HINT: Use pd.read_sql(query, engine)
cardholder_id_2_18 = pd.read_sql(query, engine)
cardholder_id_2_18.head()

,cardholder_id,date,amount
0,18,2018-01-01 23:15:10,2.95
1,18,2018-01-05 07:19:27,1.36
2,2,2018-01-06 02:16:41,1.33
3,2,2018-01-06 05:13:20,10.82
4,18,2018-01-07 01:10:54,175.00


In [4]:
# Plot for cardholder 2
cardholder_id_2 = cardholder_id_2_18[cardholder_id_2_18['cardholder_id'] == 2]
cardholder_id_2.head()
cardholder_id_2_plot = cardholder_id_2.hvplot.line('date', 'amount', label = 'Card Holder 2 Transactions', height=400, width=700, rot = 90)
cardholder_id_2_plot

:Curve   [date]   (amount)

In [5]:
# Plot for cardholder 18
cardholder_id_18 = cardholder_id_2_18[cardholder_id_2_18['cardholder_id'] == 18]
cardholder_id_18.head()
cardholder_id_18_plot = cardholder_id_18.hvplot.line('date', 'amount', label = ' Card Holder 18 Transactions', height=400, width=700, rot = 90)
cardholder_id_18_plot

:Curve   [date]   (amount)

In [6]:
# Combined plot for card holders 2 and 18
combined_plot = cardholder_id_2_plot * cardholder_id_18_plot
combined_plot

:Overlay
   .Curve.Card_Holder_2_Transactions  :Curve   [date]   (amount)
   .Curve.Card_Holder_18_Transactions :Curve   [date]   (amount)

##### Conclusion to Data Analysis Question 1

Having visualized the data, Card holder 2's spending patterns have been consistent while there are anomolous transactions on Card Holder 18 that break the holders spending patterns and would suggest fraudelent activity.

## Data Analysis Question 2

The CEO of the biggest customer of the firm suspects that someone has used her corporate credit card without authorization in the first quarter of 2018 to pay quite expensive restaurant bills. Again, for privacy reasons, you know only that the cardholder ID in question is 25.

* Using hvPlot, create a box plot, representing the expenditure data from January 2018 to June 2018 for cardholder ID 25.

* Are there any outliers for cardholder ID 25? How many outliers are there per month?

* Do you notice any anomalies? Describe your observations and conclusions in your markdown report.

In [45]:
# loading data of daily transactions from jan to jun 2018 for card holder 25
# Write the query
#use extract function to get the month value from the timestamped date

query = """
SELECT EXTRACT(month FROM transaction.date) AS month, transaction.amount
FROM transaction
JOIN credit_card ON transaction.card = credit_card.card
JOIN card_holder ON credit_card.cardholder_id = card_holder.cardholder_id
WHERE card_holder.cardholder_id = 25
AND EXTRACT(month FROM transaction.date) <= 6
ORDER BY EXTRACT(month FROM transaction.date);
        """
# Create a DataFrame from the query result. HINT: Use pd.read_sql(query, engine)
cardholder_id_25 = pd.read_sql(query, engine)
cardholder_id_25.head()
print(cardholder_id_25.tail())


    month   amount
63    6.0  1813.00
64    6.0    16.61
65    6.0    11.53
66    6.0     5.24
67    6.0     2.27


In [46]:
# loop to change the numeric month to month names

#month_name = calendar.month_name()
for x in range(cardholder_id_25.shape[0]):
    cardholder_id_25.iloc[x,0]  = calendar.month_name[int(cardholder_id_25.iloc[x,0])]

cardholder_id_25.head()


,month,amount
0,January,1.46
1,January,10.74
2,January,2.93
3,January,1.39
4,January,17.84


In [47]:
# Creating the six box plots using hvPlot
cardholder_id_25.hvplot.box(title = 'Card Holder 25 Transactions January to June 2018', y = 'amount', by = 'month', height = 400)

:BoxWhisker   [month]   (amount)

####  Conclusion to Data Analysis Question 2
Are there any outliers for cardholder ID 25? How many outliers are there per month?

Yes, after visualization of the data, we see that there are some transactions that significantly differ from Card Holder 25 usual spending every month apart from February. This can be broken down as follows:
|Month        |Number of Outliers|
|:------------|:-----------------|
|January      |1                 |
|February     |0                 |
|March        |1                 |
|April        |3                 |
|May          |1                 |
|June         |3                 |


From this, we can conclude that there is a high likelihood of fraudlent activity on Card Holder 25's corporate credit card.